In [1]:
import os
import geopandas as gpd
import pandas as pd

In [2]:
from tqdm.contrib.concurrent import process_map

In [2]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

In [3]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [4]:
enriched_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0"

In [5]:
enriched_list = os.listdir(enriched_path)

In [6]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [7]:
report_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb'

In [8]:
# skip the append, reports gdb file path
skip_list = ['appended.gdb', 'reports.gdb']

In [9]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    if f in skip_list:
        continue
    f_path = enriched_path + r"\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [10]:
# one time change for counts to mas

In [14]:
from utils.category import categorize_activity
from utils.standardize_domains import standardize_domains
from utils.counts_to_mas import counts_to_mas

In [15]:
for lyr_type in enriched_layers.keys():
    for path_dict in enriched_layers[lyr_type]:
        gdf = gpd.read_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
        #gdf = counts_to_mas(standardize_domains(categorize_activity(gdf)), 1950, 2025)
        gdf = counts_to_mas(gdf, 1950, 2025)
        #if ('BLM' in path_dict['layer_name']) or ('NFPORS' in path_dict['layer_name']) or ('NPS' in path_dict['layer_name']):
        #    print(path_dict['layer_name'])
        #    gdf.loc[gdf['ACTIVITY_START'] >= f'2023-10-01', 'COUNTS_TO_MAS'] = 'NO'
        print(path_dict['layer_name'])
        gdf.to_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
    

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-30 00:34:57,488 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-30 00:34:57,489 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-30 00:34:57,489 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-30 00:34:57,491 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-30 00:34:57,492 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-30 00:34:57,492 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-30 00:34:57,493 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category i

CNRA_enriched_20250626_point


2025-07-30 00:34:57,877 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-30 00:34:57,877 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-30 00:34:57,878 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-30 00:34:57,880 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-30 00:34:57,881 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-30 00:34:57,882 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-30 00:34:57,883 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-07-30 00:34:57,883 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


NFPORS__20250729_point


2025-07-30 00:34:58,132 INFO  [pyogrio._io           ]  Created 4,555 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-30 00:34:58,752 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-30 00:34:58,753 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-30 00:34:58,754 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-30 00:34:58,758 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-30 00:34:58,760 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-30 00:34:58,762 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-30 00:34:58,763 INFO  [utils.coun

...

USFS_Region04_enriched_20250623


2025-07-30 00:36:10,042 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-30 00:36:10,043 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-30 00:36:10,044 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-30 00:36:10,060 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-30 00:36:10,116 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-30 00:36:10,157 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-30 00:36:10,203 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-07-30 00:36:10,251 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


USFS_Region05_enriched_20250623


2025-07-30 00:36:29,935 INFO  [pyogrio._io           ]  Created 413,214 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-30 00:36:35,726 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-07-30 00:36:35,726 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-07-30 00:36:35,727 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-07-30 00:36:35,728 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-07-30 00:36:35,729 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-07-30 00:36:35,729 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-07-30 00:36:35,730 INFO  [utils.co

USFS_Region06_enriched_20250623


In [16]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-07-30 00:36:56,059 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-07-30 00:36:56,059 INFO  [process.append_polygon]  Concatenate all polygon records
2025-07-30 00:36:56,060 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250710' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\BLM_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-30 00:36:56,178 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'CNRA_enriched_20250626_polygon' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\CNRA_1950_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-07-30 00:37:00,080 INF

In [17]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [18]:
# grab timber non spatial path again
timber_nonspatial_path = None
for p in enriched_layers['point']:
    if 'Timber_Nonspatial' in p['gdb_path']:
        timber_nonspatial_path = p
        break

In [19]:
timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [20]:
%%time
for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    # drop unwanted artifact columns from California boundary df
    append_clipped = append_clipped.drop(['index_right', 'Shape_Area', 'Shape_Length'], axis=1)
    
    # industry nonspatial is by design out of california bounds and got clipped, manually concat it back
    if lyr_name == 'appended_point':
        append_clipped = pd.concat([append_clipped, timber_nonspatial], ignore_index=True)
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-07-30 00:44:24,079 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-30 00:44:24,079 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-30 00:44:24,108 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-30 00:44:53,714 INFO  [pyogrio._io           ]  Created 455,318 records
2025-07-30 00:45:00,693 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_poly
2025-07-30 00:47:28,439 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-30 00:47:28,440 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-30 00:47:28,449 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-30 00:47:48,609 INFO  [pyogrio._io           ]  Created 131,658 records
2025-07-30 00:47:52,695 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_line
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb


2025-07-30 00:47:55,653 INFO  [pyogrio._io           ]  Created 20,443 records
2025-07-30 00:47:55,803 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb appended_point


CPU times: total: 44min 28s
Wall time: 10min 19s


In [21]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [22]:
gpd.list_layers(append_path)

,name,geometry_type
0,appended_poly,MultiPolygon
1,appended_line,MultiLineString
2,appended_point,Point


In [23]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [24]:
enriched_points[(enriched_points.AGENCY =='<NA>')].PROJECTID_USER.apply(lambda x: x[:5]).unique()

array([], dtype=object)

In [25]:
enriched_polygons[(enriched_polygons.COUNTS_TO_MAS=="YES") & (enriched_polygons.ADMINISTERING_ORG.isna())]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,FIPS,STATE,ST_ABBREV,SQMI,POPULATION,geometry


In [48]:
enriched_polygons[(enriched_polygons.COUNTS_TO_MAS == 'YES') & (enriched_polygons.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()

array(['CNRA'], dtype=object)

In [39]:
#excel_out = pd.concat([enriched_points, enriched_lines, enriched_polygons])
#excel_out.drop(['geometry'], axis=1).to_csv('its_all.csv')

In [50]:
from datetime import datetime

In [51]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'DOE', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [52]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [54]:
save_gdf_to_gdb(activity_report_gdf, 
                report_path, 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-07-30 01:02:09,506 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-30 01:02:09,506 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-30 01:02:09,536 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb


2025-07-30 01:02:15,863 INFO  [pyogrio._io           ]  Created 495,290 records
2025-07-30 01:02:18,764 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\reports.gdb activity_report20250729


In [56]:
save_gdf_to_gdb(activity_report_gdf, 
                'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\activity_report_new.gdb', 
                f'activity_report20250729')

2025-07-30 01:03:37,571 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-07-30 01:03:37,571 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-07-30 01:03:37,597 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\activity_report_new.gdb


2025-07-30 01:03:43,734 INFO  [pyogrio._io           ]  Created 495,290 records
2025-07-30 01:03:46,617 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\activity_report_new.gdb activity_report20250729


In [4]:
activity_report_gdf = gpd.read_file(report_path, driver='OpenFileGDB', layer='activity_report20250729')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [57]:
activity_report_gdf.ACTIVITY_STATUS.unique()

array(['COMPLETE', 'ACTIVE'], dtype=object)

In [58]:
activity_report_gdf[activity_report_gdf.ACTIVITY_CAT.isna()]

,AGENCY,ADMINISTERING_ORG,PRIMARY_OWNERSHIP_GROUP,COUNTY,REGION,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_END,Year_txt,geometry,ENTITY_TYPE
36041,DOI,FWS,FEDERAL,KER,CENTRAL_COAST,BROADCAST_BURN,None,AGRICULTURE,COMPLETE,148.274611,AC,2016-11-07 00:00:00+00:00,2016,POINT (-119.589 35.763),Federal
36042,DOI,FWS,FEDERAL,MER,CENTRAL_COAST,BROADCAST_BURN,None,AGRICULTURE,COMPLETE,38.121273,AC,2016-12-12 00:00:00+00:00,2016,POINT (-120.633 37.179),Federal
36043,DOI,FWS,FEDERAL,MER,CENTRAL_COAST,BROADCAST_BURN,None,AGRICULTURE,COMPLETE,17.351645,AC,2016-11-07 00:00:00+00:00,2016,POINT (-120.638 37.182),Federal
36044,DOI,FWS,FEDERAL,KER,CENTRAL_COAST,BROADCAST_BURN,None,AGRICULTURE,COMPLETE,38.515092,AC,2018-10-12 00:00:00+00:00,2018,POINT (-119.582 35.764),Federal
36046,DOI,FWS,FEDERAL,MER,CENTRAL_COAST,BROADCAST_BURN,None,AGRICULTURE,COMPLETE,381.199069,AC,2018-11-09 00:00:00+00:00,2018,POINT (-120.59 37.143),Federal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37484,NPS,NPS,FEDERAL,TEH,SIERRA_NEVADA,PILE_BURN,None,FOREST,COMPLETE,1.000000,AC,2022-12-01 00:00:00+00:00,2022,POINT (-121.613 40.345),Federal
37485,NPS,NPS,FEDERAL,TEH,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,COMPLETE,13.000000,AC,2023-06-28 00:00:00+00:00,2023,POINT (-121.611 40.347),Federal
37493,NPS,NPS,FEDERAL,HUM,NORTH_COAST,BROADCAST_BURN,None,FOREST,COMPLETE,854.000000,AC,2022-10-08 00:00:00+00:00,2022,POINT (-123.874 41.138),Federal
37497,NPS,NPS,FEDERAL,SHA,SIERRA_NEVADA,BROADCAST_BURN,None,FOREST,COMPLETE,225.000000,AC,2023-09-27 00:00:00+00:00,2023,POINT (-121.568 40.524),Federal


In [60]:
activity_report_gdf[activity_report_gdf.Year_txt == '2024'].groupby(['AGENCY', 'ADMINISTERING_ORG', 'ACTIVITY_STATUS']).sum('ACTIVITY_QUANTITY')

ACTIVITY_QUANTITY
AGENCY ADMINISTERING_ORG ACTIVITY_STATUS                   
CALSTA CALTRANS          COMPLETE             133459.499800
CNRA   CALFIRE           ACTIVE               138469.461500
                         COMPLETE              33542.742500
       CCC               ACTIVE                  261.190000
                         COMPLETE               1522.270000
       CDFW              COMPLETE             103877.059607
       PARKS             COMPLETE              15087.253321
       SDRC              ACTIVE                 3842.530000
                         COMPLETE                131.160000
       SMMC              COMPLETE                618.066657
       SNC               ACTIVE                10441.770000
                         COMPLETE               1802.690000
       TAHOE             ACTIVE                  250.500000
                         COMPLETE                 54.000000
       WCB               ACTIVE                 1301.550000
                         COMPLETE               6369.910000
DOD    DOD               COMPLETE               2967.330000
DOE    DOE               COMPLETE               5482.900000
DOI    BIA               COMPLETE                 69.000000
       BLM               COMPLETE              40467.823381
       FWS               COMPLETE              19255.000000
NPS    NPS               COMPLETE               3547.000000
TIMBER TIMBER            COMPLETE             186690.412376
USDA   USFS              COMPLETE             334071.200000

In [49]:
activity_report_gdf[activity_report_gdf.Year_txt == '2022'].groupby(['AGENCY', 'ADMINISTERING_ORG']).sum('ACTIVITY_QUANTITY')

ACTIVITY_QUANTITY
AGENCY ADMINISTERING_ORG                   
CALSTA CALTRANS               122514.696340
CNRA   CALFIRE                160727.116890
       CCC                      1117.083820
       CDFW                   108346.846762
       PARKS                   12814.796613
       RMC                        79.470000
       SCC                       891.150000
       SDRC                      317.920000
       SMMC                      499.324782
       TAHOE                     310.700000
       WCB                      3509.710000
DOD    DOD                      4667.000000
DOE    DOE                      4412.690000
DOI    BIA                      7401.000000
       BLM                     27152.304505
       FWS                     52991.000000
NPS    NPS                      3609.000000
TIMBER TIMBER                 222278.078178
USDA   USFS                   142327.600000

In [30]:
activity_report_gdf[(activity_report_gdf.ACTIVITY_DESCRIPTION == "AMW_AREA_RESTOR") | (activity_report_gdf.ACTIVITY_DESCRIPTION == "OAK_WDLND_MGMT")]

,AGENCY,ADMINISTERING_ORG,PRIMARY_OWNERSHIP_GROUP,COUNTY,REGION,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_END,Year_txt,geometry,ENTITY_TYPE
131462,CNRA,CDFW,STATE,MAD,SIERRA_NEVADA,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,3.680000,AC,2022-01-31 00:00:00+00:00,2022,POINT (56710.253 -88969.102),State
20390,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,147.163751,AC,2021-12-31 00:00:00+00:00,2021,POINT (-284207.678 -48707.918),Timber Companies
20391,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,280.000000,AC,2022-12-31 00:00:00+00:00,2022,POINT (-284207.678 -48707.918),Timber Companies
20392,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,14.327502,AC,2023-12-31 00:00:00+00:00,2023,POINT (-284207.678 -48707.918),Timber Companies
20393,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,0.000000,AC,2024-12-31 00:00:00+00:00,2024,POINT (-284207.678 -48707.918),Timber Companies
4574,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,5.170592,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.03 40.335),State
4575,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,0.241517,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.034 40.337),State
4577,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,4.860597,AC,2023-08-25 00:00:00+00:00,2023,POINT (-124.039 40.34),State
4580,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,1.676779,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.032 40.336),State
5427,CNRA,PARKS,STATE,HUM,NORTH_COAST,OAK_WDLND_MGMT,MECH_HFR,FOREST,COMPLETE,2.321638,AC,2023-08-24 00:00:00+00:00,2023,POINT (-124.033 40.337),State
